In [ ]:
#|default_exp utils.misc

# Miscellaneous

> Miscellaneous functions.

In [ ]:
#|export
from fastcore.all import *
import numbers
import polvo as pv

from PIL import Image

/home/lgvaz/git/polvo/polvo/utils/logging.py:8: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [ ]:
#|export
def kwargs_grid(**kwargs):
    "Returns a generator with all combinations of kwargs"
    return (dict(zip(kwargs.keys(), v)) for v in itertools.product(*kwargs.values()))

In [ ]:
list(kwargs_grid(size=[(224, 224), (512, 512)], alpha=[0.3, 0.7, 0.9], apply_tfms=[False, True]))

[{'size': (224, 224), 'alpha': 0.3, 'apply_tfms': False},
 {'size': (224, 224), 'alpha': 0.3, 'apply_tfms': True},
 {'size': (224, 224), 'alpha': 0.7, 'apply_tfms': False},
 {'size': (224, 224), 'alpha': 0.7, 'apply_tfms': True},
 {'size': (224, 224), 'alpha': 0.9, 'apply_tfms': False},
 {'size': (224, 224), 'alpha': 0.9, 'apply_tfms': True},
 {'size': (512, 512), 'alpha': 0.3, 'apply_tfms': False},
 {'size': (512, 512), 'alpha': 0.3, 'apply_tfms': True},
 {'size': (512, 512), 'alpha': 0.7, 'apply_tfms': False},
 {'size': (512, 512), 'alpha': 0.7, 'apply_tfms': True},
 {'size': (512, 512), 'alpha': 0.9, 'apply_tfms': False},
 {'size': (512, 512), 'alpha': 0.9, 'apply_tfms': True}]

In [ ]:
#|export
@functools.wraps(zip)
def safe_zip(*args, **kwargs):
    if len(set(map(len, args))) not in (0, 1):
        raise ValueError(f'All elements should have the same size, but got {[len(x) for x in args]}')
    return zip(*args, **kwargs)

In [ ]:
test_eq(list(safe_zip([1, 2], [3, 4], [5, 6])), list(zip([1, 2], [3, 4], [5, 6])))

In [ ]:
test_fail(lambda: safe_zip([1, 2], [3, 4], [5]), contains='same size')

In [ ]:
#|export
def skip_error(fn):
    'Returns the error instead of raising it.'
    def _inner(*args, **kwargs):
        try: return fn(*args, **kwargs)
        except Exception as e: return e
    return _inner

In [ ]:
skip_error(lambda: 'test'.get(2))()

AttributeError("'str' object has no attribute 'get'")

In [ ]:
# FIND CORRECT PLACE TO PLACE THIS
def wandb_upload(project:str, artifact_name:str, *path:str, type='dataset'):
    "Uploads files or dir to wandb"
    import wandb
    run = wandb.init(project=project, job_type="data-upload")
    artifact = wandb.Artifact(artifact_name, type=type)
    
    def _add_file(p): artifact.add_file(p, p)
    for p in path:
        if Path(p).is_file(): _add_file(p)
        else:
            for f in pv.get_files(p): _add_file(f)

    run.log_artifact(artifact)

In [ ]:
class ParameterIterator:
    def __call__(self, f):
        sig = inspect.signature(f)
        @functools.wraps(f)
        def wrapper(*args, **kwargs):
            bound = sig.bind(*args, **kwargs)
            for k, v in bound.arguments.items():
                self.apply(k, v)
            return f(*args, **kwargs)
        return wrapper

In [ ]:
def _save_image(image, save_dir): return pv.save_image(image, save_dir)

class save_params(ParameterIterator):
    _save_fns = {
        Image: pv.save_image,
        np.ndarray: pv.save_image,
        numbers.Number: lambda s: pv.save_txt(s+'\n', append=True),
        str: lambda s: pv.save_txt(s+'\n', append=True),
    }
    
    def __init__(
        self,
        save_dir,
        save_fns=None, # Dictionary of {<type>: save_fn}. `save_fn` first parameter should be the object, and second the path.
    ):
        "Save all parameters of decorated function"
        self.save_dir = pv.mkdir(save_dir, exist_ok=True)
        self.save_fns = save_fns or self._save_fns.copy()
        
    def apply(self, k, v):
        save_fns[type(k)](v, self.save_dir)

In [ ]:
# @save_params('foo_params')
# def foo(a, b, c=2, d='test'):
#     return

In [ ]:
#|hide
from nbdev import nbdev_export
nbdev_export()